# Query for User Information

In [1]:
import requests
import pickle
import pandas as pd
import urllib
import time

In [2]:
user_align=pickle.load(open('Data/user_alignments.pkl'))
user_align=pd.DataFrame(user_align.items(),columns=['user','align'])

In [3]:
def not_ip(s):
    a = s.split('.')
    if len(a) != 4:
        return True
    for x in a:
        if not x.isdigit():
            return True
        i = int(x)
        if i < 0 or i > 255:
            return True
    return False

In [4]:
users=[u for u in user_align['user'].values if not_ip(u)]

In [ ]:
user_info=[]
i=0
chunksize=50

In [ ]:
while i*chunksize<=len(users):
    u=users[i*chunksize:(i+1)*chunksize]
    u='|'.join(u)
    query="http://en.wikipedia.org/w/api.php?action=query&list=users&ususers={}&usprop=editcount&format=json".format(urllib.quote(u, safe='|'))
    resp=requests.get(query)
    if resp.ok:
        js=resp.json()
        user_info.extend(js['query']['users'])
    i+=1
    time.sleep(0.5)

In [21]:
clean_user=[i for i in user_info if 'invalid' not in i]

In [35]:
df=pd.io.json.json_normalize(clean_user)

In [37]:
del df['missing']

In [38]:
df.dropna(inplace=True)

In [42]:
df.to_csv('Data/user_edits.tsv',sep='\t',index=False, encoding='utf-8')